<h2>References</h2>

https://discordpy.readthedocs.io/en/stable/api.html

https://discordpy.readthedocs.io/en/stable/ext/commands/api.html#

# 0. Python Review

## 0.1 Lists

In [ ]:
# Creating a list
sports = ['Soccer', 'Basketball', 'Baseball', 'Hockey']
numbers = [1,2,3,4,5,6]

# Adding items
sports.append('Swimming')
sports.extend(['Curling', 'Fooseball'])

# Iterating through lists
for i,sport in enumerate(sports):
  print(f'Item {i+1}: {sport}')

# List comprehensions
numbers = [i+1 for i,sport in enumerate(sports)]
print(numbers)

# Indexing



Item 1: Soccer
Item 2: Basketball
Item 3: Baseball
Item 4: Hockey
Item 5: Swimming
Item 6: Curling
Item 7: Fooseball
[1, 2, 3, 4, 5, 6, 7]
[7, 6, 5, 4, 3, 2, 1]


## 0.2 Maps

In [ ]:
# Creating a map
map = {}

# Storing key,value pairs
map['key'] = 'value'
map[5] = 'another_value'

# Checking if a key exists
my_keys = map.keys()
print(f'My Keys: {my_keys}')
if 5 in map.keys():
  print('The key exists')
else:
  print('The key does not exist')

# Iterating through k,v pairs (items)
# Removing keys

del map

## 0.3 Nested Structures

In [ ]:
# List of Lists, Dictionary with Lists, etc.

import json
my_map = {}
KEY = 'LISTS'
my_map[KEY] = sports
metadata = {}
metadata['timestamp'] = "12:05pm"
metadata['url'] = 'https://google.com'
metadata['name'] = 'Google'
sports.append(metadata)
printable = json.dumps(my_map, indent=4)
print(printable)
url = my_map['LISTS'][7]['url']
print(url)

{
    "LISTS": [
        "Soccer",
        "Basketball",
        "Baseball",
        "Hockey",
        "Swimming",
        "Curling",
        "Fooseball",
        {
            "timestamp": "12:05pm",
            "url": "https://google.com",
            "name": "Google"
        }
    ]
}
https://google.com


## 0.4 Object Oriented (IMPORTANT!!!!)

In [ ]:
# Define member class
class Member:
  def __init__(self, name: str, age: int):
    self.name = name
    self.age = age
  
  def __str__(self):
    return f'(Name: {self.name} , Age: {self.age})'

# Define some members
mem1 = Member('Joe', 21)
mem2 = Member('Julio', 18)
mem3 = Member('Mike', 5)
mem4 = Member('Sally', 22)
member_list = [mem1,mem2,mem3,mem4]

# Define the server
class DiscordServer:
  def __init__(self, name: str, ID: int):
    self.name = name
    self.ID = ID
    self.members = []
  
  def addMember(self, member: Member):
    self.members.append(member)

  def __str__(self):
    output = ""
    output += self.name + '\n'
    for mem in self.members:
      output += str(mem) + '\n'
    return output

# Create the server
Guild = DiscordServer("GDSC Club", 123)

# Add our members
for mem in member_list:
  Guild.addMember(mem)

print(Guild)

GDSC Club
(Name: Joe , Age: 21)
(Name: Julio , Age: 18)
(Name: Mike , Age: 5)
(Name: Sally , Age: 22)



## 0.5 An Aside about Lambda Functions

In [ ]:
# sorted
nums = [100,-5,2,-70]
s = sorted(nums)
print(s)
print('')

# lambda functions
func1 = lambda x: x + 1
func2 = lambda mem1,mem2: mem1.age < mem2.age
print(func1(10))
print(func2(mem1,mem2))

# Useful for working with objects
sorted_members = sorted(Guild.members, key=lambda m: m.age)
for mem in sorted_members:
  print(str(mem))

[-70, -5, 2, 100]

11
False
(Name: Mike , Age: 5)
(Name: Julio , Age: 18)
(Name: Joe , Age: 21)
(Name: Sally , Age: 22)


# 1. Preliminaries

## 1.1 Setup Discord
<p>
  <ul>
    <li>Create Discord account, confirm email, set developer mode on in settings</li>
    <li>Create a server</li>
    <li>Go to discord.com/developers/ and create an application</li>
    <li>Create a bot, declare intents, add to server</li>
    
  </ul>
</p>


## 1.2 Install Dependencies

In [ ]:
# Install using pip (Python Package Manager)
!pip install discord.py
!pip install colab-env --upgrade
!pip install tabulate

## 1.3 Setup Google Drive Access

In [ ]:
# Give Colab Access to Google Drive
from google.colab import drive
drive.mount('/content/drive/')

##1.4 Imports

In [ ]:
# Import the necessary modules/libraries we'll need
import discord
from discord.ext import commands
import colab_env
import os, re, requests, random, asyncio, json
from tabulate import tabulate

# 1.5 Setup Environment Variables (Discord Token)


In [ ]:
# Add discord token as environment variable
colab_env.envvar_handler.add_env('TOKEN', '***', overwrite=True)

In [ ]:
# Terminal commands to check out the files in our drive
!ls

In [ ]:
!cat gdrive/MyDrive/vars.env

# 2. Common Objects We'll Work With

<h3>TextChannel</h3>

---

<ul>
  <li>id</li>
  <li>name</li>
  <li>last_message</li>
  <li>members</li>
  <li>history(limit)</li>
  <li>send(content)</li>
</ul>

<h3>Member</h3>

---

<ul>
  <li>id</li>
  <li>name</li>
  <li>history(limit)</li>
  <li>send(content)</li>
  <li>kick(reason)</li>
  <li>ban()</li>
  <li>unban(reason)</li>
</ul>

<h3>Message</h3>

---

<ul>
  <li>author</li>
  <li>channel</li>
  <li>add_reaction(emoji)</li>
  <li>reply(content)</li>
</ul>

<h3>Context (Command Based)</h3>

---

<ul>
  <li>guild</li>
  <li>channel</li>
  <li>message</li>
  <li>send(content)</li>
  <li>reply(content)</li>
</ul>

<h3>Utility Functions</h3>

---

<ul>
  <li>discord.utils.find(predicate, iterable)</li>
  <li>discord.utils.get(iterable, **attrs)</li>
</ul>

# 3. Let's Get Our Bot Running!!

### Tenor Setup

In [ ]:
# PUBLIC TENOR API KEY
PUB_TENOR_KEY = 'LIVDSRZULELA'

# Define the search parameters
query = 'welcome'
query = query.replace(' ', '+')
limit = 5

# Send the http request
request = f'https://g.tenor.com/v1/search?q={query}&key={PUB_TENOR_KEY}&limit={limit}'
response = requests.get(request)

# Process the http response
if response.status_code == 200:
  results = response.json()['results']
  chosen_result = random.choice(results)
  gif_url = chosen_result['url']
  print(gif_url)

https://tenor.com/bmRRR.gif


In [ ]:
# How to get to a gif url
response.json()['results'][0]['url']

'https://tenor.com/5PGb.gif'

### Stipop Setup

In [ ]:
# STICKER API KEY
DEMO_STI_KEY = "***"

# Define the search parameters
query = 'query'
query = query.replace(' ', '+')
limit = 5
url = f'https://messenger.stipop.io/v1/search?userId={1}&q={query}&lang=en&pageNumber={1}&limit={limit}'

# Send the http request
response = requests.get(url, headers={'apikey' : DEMO_STI_KEY})

# Process the http response
if response.status_code == 200:
  sticker_results = response.json()['body']['stickerList']
  sticker_url = random.choice(sticker_results)['stickerImg']
else:
  print('Sorry, could not find the sticker.')

In [ ]:
# How to get to a sticker url
response.json()['body']['stickerList'][0]['stickerImg']

'https://img.stipop.io/2020/10/19/1603448719388_19.png'

###Discord Setup

In [ ]:
# Get our token
TOKEN = os.getenv('TOKEN')

# Give Discord our intentions
intents = discord.Intents.all()

# Define our client
client = commands.Bot(intents=intents, command_prefix='$')

del map
map = {}

## 3.1 Define how the bot should operate

### Event: On Ready

In [ ]:
# Note:
# Remember to decorate each function!!!

# First Bot Event
# Send a message when the bot connects to Discord
@client.event
async def on_ready():
  general = client.get_channel(899019412525445184)
  await general.send("Connected to Discord!!")

### Run the bot

In [ ]:
# Note: In Colab or a Jupyter Notebook, regular client.run() 
# does not work due to conflict with the Notebook event loop and thus
# need to use asyncio to create a COROUTINE on a separate event loop
# (client.run internally calls start, and creates the loop whereas here we need to directly call start)
task = asyncio.get_event_loop().create_task(client.start(TOKEN)) 

### Stop the bot

In [ ]:
task.cancel()

False

### Event: On Member Join

In [ ]:
# WELCOME BOT
# Send a GIF and a welcome message to newly joined members
@client.event
async def on_member_join(member):
  # Make the HTTP request
  response = requests.get(request)
  if response.status_code == 200:
    choice = random.randint(0,limit-1)
    results = response.json()['results']
    chosen_result = results[choice]
    gif_url = chosen_result['url']
    # Send content to newly joined member
    await member.send(gif_url)
  await member.send(f'Hello {member.name}! Welcome to the club!')

### Event: On Message

In [ ]:
# Let's define some bad words for our moderator to detect (AFTER EXPLAINING RegEx)
bad_words = ['pickle', 'tomato', 'oracle']
bad_words = words_to_expressions(bad_words)
bad_words

In [ ]:
# MODERATOR EVENTS 
# (NEED TO CALL await client.process_commands(message) at the end OTHERWISE CAN'T ISSUE COMMANDS)
# Listen for messages
@client.event
async def on_message(message):
  # We don't want to call this on our own messages (infinite feedback loop == bad)
  if message.author == client.user:
    return

  # Let's moderate!
  found = [re.search(word, message.content.lower()) for word in bad_words]
  if any(found):
    # Keep track of the offending user's strikes
    if message.author in map.keys():
      map[message.author] += 1
    else:
      map[message.author] = 1
    # If they exceed 3 strikes, that's a ban/kick or whatever it may be
    if map[message.author] >= 3:
      await message.channel.send(f'Sorry {message.author}, you\'re outta here.')
      await message.author.kick(reason="Violated community guidelines.")
    else:
      await message.channel.send(f"{message.author}, please refrain from such language!\nStrikes: {map[message.author]}")
  await client.process_commands(message)
  # # Send message to channel, to check that we can read messages that are sent
  # await message.channel.send("Cool, I can read the chat!")

### Command: Get Strikes

In [ ]:
# Get Strikes Command
# $strikes

@client.command()
async def strikes(ctx):
  headers = ["User", "Strikes", "Status"]
  records = []
  for k,v in map.items():
    user = discord.utils.find(lambda m: m == k, ctx.guild.members)
    # Continue if some problem finding the user
    if user == None:
        continue
    # Check status
    if v == 0:
      status = "Great"
    elif v == 1:
      status = "Okay"
    elif v == 2:
      status = "Danger"
    else:
      status = "Kicked"
    records.append([f"{user.name}", f"{map[user]}", f"{status}"])
  table = tabulate(records, headers=headers, tablefmt="github", stralign="center", numalign="center")
  await ctx.send(table)

### Command: Send a Gif

In [ ]:
# Gif Command
# $gif

@client.command()
async def gif(ctx, search):
  await ctx.send('Attempting to find gif')
  search = search.replace(' ', '+')
  request = f'https://g.tenor.com/v1/search?q={search}&key={PUB_TENOR_KEY}&limit={limit}'
  response = requests.get(request)
  if response.status_code == 200:
    choice = random.randint(0,limit-1)
    results = response.json()['results']
    chosen_result = results[choice]
    gif_url = chosen_result['url']
    await ctx.send(gif_url)
  else:
    await ctx.send('Could not find a gif for that search')

### Command: Send a Sticker

In [ ]:
# Sticker Command
# $sticker [search]

@client.command()
async def sticker(ctx, search):
  await ctx.send('Attempting to find sticker')
  query = search.replace(' ', '+')
  url = f'https://messenger.stipop.io/v1/search?userId={1}&q={query}&lang=en&pageNumber={1}&limit={1}'
  my_key = "39450ba1a2a3803704e956e3f08643f4"
  r = requests.get(url, headers={'apikey' : my_key})
  await ctx.send('Made the http request')
  if r.status_code == 200:
    sticker_url = r.json()['body']['stickerList'][0]['stickerImg']
    await ctx.send(sticker_url)
  else:
    await ctx.send('Sorry, could not find the sticker.')

# 4. Text Parsing With Regular Expressions
<h3>Python 're' module</h3>

---

<ul>
  <li>search(pattern, text)  -> Match Object</li>
  <li>findall(pattern, text) -> List[str]</li>
  <li>split(pattern, text) -> List[str]</li>
</ul>


## 3.1 Motivation for RegEx

In [ ]:
# Define a regular expression for pattern matching
# Search a text for a word
words = ["pickle", "orange"] # some bad words
bad_word = words[0]
text = "i think pickles are good?" # some text to parse
found = re.search(bad_word, text)
found.group(0)

'pickle'

In [ ]:
# What if there is some variance in the text?
text = "i think p i c k l e s are good?"
found = re.search(bad_word, text)
if found:
  print(found.group(0))
else:
  print("seems like we didn't find any of the bad words, hmmmm")

seems like we didn't find any of the bad words, hmmmm


In [ ]:
text = "i think p_i_c_k_l_e_s are good?"
"""
'.'  : any character
'\s' : whitespace characters
'\d' : digits
'\w' : unicode character
'*'  : match the previous character zero or more times
'+'  : match the previous character one or more times
'()' : grouping
'[]' : match any one of the characters enclosed
"""
pattern = r".*p.*i.*c.*k.*l.*e.*s.*"
found = re.search(pattern, text)
found.group(0)

'i think p_i_c_k_l_e_s are good?'

In [ ]:
# Algorithm1:
# 1. Create an empty character array
# Loop:
#   2. Append the letter
#   3. Append the .*
# 4. join the character array as a single string with no separator
def words_to_expresions_c_style(words):
  expressions = []
  for word in words:
    char_list = []
    for letter in word:
      char_list.append(letter)
      char_list.append('.')
      char_list.append('*')
    expression = ""
    for letter in char_list:
      expression += letter
    expressions.append(expression)
    print(f"From algorithm1: {expression}")
  return expressions
  
# Algorithm2:
# 1. Split the word into it's character array -> [letter for letter in word]
# 2. Join character array as string, separate the characters by .*
def words_to_expressions(words):
  expressions = []
  for word in words:
    pattern = ".*".join(list(word))
    print(f"From algorithm2: {pattern}")
    expressions.append(pattern)
  return expressions
words_to_expresions_ver_1(words)
print()
expressions = words_to_expressions(words)

From algorithm1: p.*i.*c.*k.*l.*e.*
From algorithm1: o.*r.*a.*n.*g.*e.*

From algorithm2: p.*i.*c.*k.*l.*e
From algorithm2: o.*r.*a.*n.*g.*e


In [ ]:
expressions

['p.*i.*c.*k.*l.*e', 'o.*r.*a.*n.*g.*e']

In [ ]:
# We have our text
text = "i think oranges are good?"
# call re.search on every expression in expressions and store as list
found = [re.search(expr, text) for expr in expressions]
print(found)

[None, <re.Match object; span=(8, 19), match='oranges are'>]


In [ ]:
# any(iterable) -> returns True if any of the elements are True (None == False, 0 == False, False == False)
if any(found):
  print("We found a matching expression in the text")
else:
  print("The text is clean!")

We found a matching expression in the text


In [ ]:
# Case Study System Path Simplification (if there is interest) (FAANG Interview Question)
"""
Given an abolute file path:
  Find the canonical path

In a Unix-style file system, a period '.' refers to the current directory, 
a double period '..' refers to the directory up a level, 
and any multiple consecutive slashes (i.e. '//') are treated as a single slash '/'. 

Input: path = "/a/./b/../../c/"
Output: "/c"
"""

'\nGiven an abolute file path:\n  Find the canonical path\n\nIn a Unix-style file system, a period \'.\' refers to the current directory, \na double period \'..\' refers to the directory up a level, \nand any multiple consecutive slashes (i.e. \'//\') are treated as a single slash \'/\'. \n\nInput: path = "/a/./b/../../c/"\nOutput: "/c"\n'